In [250]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter

import torch
import torch.nn.functional as f
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from random import random
from tqdm.notebook import tqdm

# データのロード

In [7]:
WORK_DIR = Path.cwd()
path_train_csv = WORK_DIR/"train.csv/"
train_csv = pd.read_csv(path_train_csv)
path_train_img = WORK_DIR/"train_images/"

# データのペアの作成
二つの画像が同じ商品かを判定するための画像のペアを作成する。<br>
match_rateの比率に基づいて一致する。

In [186]:
match_rate = 0.3
paired_num_list = []
match_or_not_list = []
train_csv_paired = train_csv.copy()
for i in range(len(train_csv)):
    df = train_csv.iloc[i]
    posting_id = df.posting_id
    label_group = df.label_group
    if random() < match_rate:
        #同じlabel_groupの画像同士を用いる
        df_samelabel = train_csv[(train_csv.label_group == label_group)
                                 &
                                 (train_csv.posting_id != posting_id)]
        df2 = df_samelabel.sample().copy()
        is_match = 1
    else:
        #異なるlabel_groupの画像同士を用いる
        df2 =  train_csv[train_csv.index != posting_id].sample().copy()
        is_match = 0
    paired_num_list.append(df2.index[0]) #ペアにした画像のindexを加える
    match_or_not_list.append(is_match) #ペアの画像のlabel_groupがマッチするかを加える
train_csv_paired.loc[:,["paired_num"]] = paired_num_list
train_csv_paired.loc[:,["match_or_not"]] = match_or_not_list 

# Dataset, Dataloader

In [210]:
#==Dataset==
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, df, dir_image, input_size, transform = None):
        self.transform = transform
        self.df = df
        self.dir_image = dir_image 
        self.input_size = input_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #画像を2枚と、その2枚の画像のlbael_groupが一致(1)か不一致(0)かを出力する。
        #一枚目の画像はidxで指定し、2枚目はランダムで指定
        df = self.df.iloc[idx]
        imagename = df.image 
        path_image = self.dir_image/imagename
        image = Image.open(path_image)
        image = image.resize(self.input_size)
        image = torchvision.transforms.functional.to_tensor(image)
        
        paired_num = df.paired_num
        df2 = self.df.iloc[paired_num]
        imagename2 = df2.image
        path_image2 = self.dir_image/imagename2
        image2 = Image.open(path_image2)
        image2 = image2.resize(self.input_size)
        image2 = torchvision.transforms.functional.to_tensor(image2)
        
        match_or_not = torch.tensor(df.match_or_not)
            
        return image,image2,match_or_not
    
    
#==Dataloader==
train_dataset = Mydatasets(train_csv_paired,WORK_DIR/"train_images/",(224,224))
train_dataloader = DataLoader(
    train_dataset, batch_size=16, shuffle=True,
    #num_workers=2, 
    drop_last=True)

images, images2, labels = next(iter(train_dataloader))

# モデル 

In [211]:
from torchvision import models
res = models.resnet18(pretrained=True)
# すべてのパラメータを固定
for param in res.parameters():
    param.requires_grad = False

In [234]:
class Net(torch.nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.fc1_1 = torch.nn.Linear(1000, 300)
            self.fc1_2 = torch.nn.Linear(300, 50)
            self.fc2_1 = torch.nn.Linear(1000, 300)
            self.fc2_2 = torch.nn.Linear(300, 50)
            self.fc3_1 = torch.nn.Linear(100,50)
            self.fc3_2 = torch.nn.Linear(50,2)
        def forward(self,x1,x2):
            x1 = res(x1)
            x1 = self.fc1_1(x1)
            x1 = torch.sigmoid(x1)
            x1 = self.fc1_2(x1)
            
            x2 = res(x2)
            x2 = self.fc2_1(x2)
            x2 = torch.sigmoid(x2)
            x2 = self.fc2_2(x2)
            
            x = torch.cat([x1, x2], axis=1)
            x = self.fc3_1(x)
            x = self.fc3_2(x)
            
            return f.log_softmax(x, dim=1)

# 学習

In [235]:
model = Net()

In [236]:
epoch = 20
# 学習結果の保存用
history = {
        'train_loss': [],
        'test_loss': [],
        'test_acc': [],
    }
optimizer = torch.optim.Adam(params=net.parameters(), lr=0.001)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
for e in tqdm(range(epoch)):
    loss = None

    model.train(True)
    
    for i, (image,image2,label_group) in tqdm(enumerate(train_dataloader)):
        outputs = model(image,image2)
        loss = criterion(outputs, label_group)
        loss.backward()
        optimizer.step()
    # 学習のストップ
    net.eval()  # または net.train(False) でも良い

  0%|          | 0/20 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]